# Title

In [77]:
from csv import reader

In [78]:
opened_file = open("/home/dq/notebook/AppleStore.csv")
read_file = reader(opened_file)
apple_list = list(read_file)

opened_file = open("/home/dq/notebook/googleplaystore.csv")
read_file = reader(opened_file)
android_list = list(read_file)

Function: explore_data

Displays rows of a dataset.

In [79]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Function: convert_float

Removes any non-number characters from a string and converts it to a float.
Some of the ratings and price fields were returning errors due to non-number values, so I made this function to remove any and return just numbers and decimal places. It loops through string, checks each character, and adds it to ret_string, which is then returned. If ret_string is empty because it had no number characters, it sets ret_string to 0.

In [80]:
def convert_float(string):
    ret_string = ''
    for char in string:
        if ord(char) > 45 and ord(char) < 58 and char != '/':
            ret_string += char
        
    if ret_string == '':
        ret_string = '0'
    return float(ret_string)

Function: is_english

Checks whether a string has more than 3 non-English characters.
Loops through string, checking each character and incrementing i if it is not within the acceptable range. When i gets above 3, a False value is returned, otherwise it returns True.

In [81]:
def is_english(string):
    i = 0
    for char in string:
        if ord(char) > 127:
            i += 1
            if i > 3:
                return False
    return True

Function: find_max_reviews

Returns a dictionary containing the maximum number of reviews for duplicate rows.
Creates a dictionary called reviews_max, where each app name and number of reviews are added if the name is not in the dictionary, or updated if the number of reviews is the highest so far. At the end, reviews_max is returned.

In [82]:
def find_max_reviews(dataset, name_pos, review_pos):
    reviews_max = {}

    for row in dataset:
        name = row[name_pos]
        n_reviews = convert_float(row[review_pos])
        if name in reviews_max and reviews_max[name] < n_reviews:
            reviews_max[name] = n_reviews
        if name not in reviews_max:
            reviews_max[name] = n_reviews

    return reviews_max

Function: remove_duplicates

Removes duplicates from a dataset.
Uses the dictionary created in find_max_reviews to identify the rows with the highest number of reviews. Loops through dataset and adds row to list_clean if the name is in the dictionary and it hasn't already been added. After list_clean has one row for each app name, it is returned.

In [83]:
def remove_duplicates(dataset, name_pos, review_pos):
    list_clean = []
    already_added = []
    reviews_max = find_max_reviews(dataset, name_pos, review_pos)

    for row in dataset:
        name = row[name_pos]
        n_reviews = convert_float(row[review_pos])
        if n_reviews == reviews_max[name] and name not in already_added:
            list_clean.append(row)
            already_added.append(name)
        
    print('\n')
    print(len(dataset) - len(list_clean), ' duplicate rows removed.')
    print(len(list_clean), ' rows remaining.')
    print('\n')
    
    return list_clean

Function: remove_non_english

Removes rows with more than 3 non-English characters.
Uses is_english to check the app name, adds it to list_clean if it returns True, and returns list_clean.

In [84]:
def remove_non_english(dataset, name_pos):
    list_clean = []
    for row in dataset:
        if is_english(row[name_pos]):
            list_clean.append(row)
    
    print('\n')
    print(len(dataset) - len(list_clean), ' non-English rows removed.')
    print(len(list_clean), ' rows remaining.')
    print('\n')
    
    return list_clean

Function: remove_non_free

Removes rows with prices other than 0.
Uses a similar loop to check the price and add the row to list_clean. My convert_float function is used to remove non-number characters from the price.

In [85]:
def remove_non_free(dataset, name_pos, price_pos):
    list_clean = []
    for row in dataset:
        name = row[name_pos]
        price = convert_float(row[price_pos])
        if price == 0:
            list_clean.append(row)
    
    print('\n')
    print(len(dataset) - len(list_clean), ' non-free rows removed.')
    print(len(list_clean), ' rows remaining.')
    print('\n')
    
    return list_clean

These functions create the frequency tables for analyzing the data.

In [86]:
def freq_table(dataset, index):
    freq_dict = {}
    
    for row in dataset:
        key = row[index]
        if key not in freq_dict:
            freq_dict[key] = 1
        else:
            freq_dict[key] += 1
    
    return freq_dict

In [87]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [88]:
def user_table(dataset, user_index, genre_index):
    user_dict = {}
    unique_genres = freq_table(dataset, genre_index)
    
    for genre in unique_genres:
        total = 0
        len_genre = 0
        for row in dataset:
            genre_app = row[genre_index]
            if genre_app == genre:
                total += convert_float(row[user_index])
                len_genre += 1
            
        avg_users = total / len_genre
        user_dict[genre] = avg_users
    
    return user_dict

Before cleaning the data, we can use explore_data to display the first few rows in each dataset.

These are the first few rows of the Apple dataset:

In [89]:
explore_data(apple_list, 1, 5, 'true')

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7198
Number of columns: 16


These are the first few rows of the Android dataset:

In [90]:
explore_data(android_list, 1, 5, 'true')

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10842
Number of columns: 13


In [91]:
print("Apple Column Names:\n")
explore_data(apple_list, 0, 1)
print("\nAndroid Column Names:\n")
explore_data(android_list, 0, 1)

Apple Column Names:

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']



Android Column Names:

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']




Both the Google Play and the Apple datasets have duplicate rows.

These are all the rows in the Apple dataset with duplicates:

In [92]:
duplicate = []
unique = []

for row in apple_list[1:]:
    name = row[1]
    if name in unique:
        duplicate.append(name)
        for row2 in apple_list[1:]:
            if row2[1] == name:
                print(row2)
    else:
        unique.append(name)

['1173990889', 'Mannequin Challenge', '109705216', 'USD', '0.0', '668', '87', '3.0', '3.0', '1.4', '9+', 'Games', '37', '4', '1', '1']
['1178454060', 'Mannequin Challenge', '59572224', 'USD', '0.0', '105', '58', '4.0', '4.5', '1.0.1', '4+', 'Games', '38', '5', '1', '1']
['952877179', 'VR Roller Coaster', '169523200', 'USD', '0.0', '107', '102', '3.5', '3.5', '2.0.0', '4+', 'Games', '37', '5', '1', '1']
['1089824278', 'VR Roller Coaster', '240964608', 'USD', '0.0', '67', '44', '3.5', '4.0', '0.81', '4+', 'Games', '38', '0', '1', '1']


In [93]:
print('Number of Duplicates:', len(duplicate))
print('Expected length without the duplicates:',
      len(apple_list[1:]) - len(duplicate))

Number of Duplicates: 2
Expected length without the duplicates: 7195


Now, we will run the functions to clean up the data for the Apple dataset.

First, we will remove the duplicate rows. We will keep only the row with the most reviews, because that should be the most recent entry.

In [94]:
apple_clean = remove_duplicates(apple_list[1:], 1, 6)



2  duplicate rows removed.
7195  rows remaining.




Now, we will remove rows for the non-English apps.

In [95]:
apple_clean = remove_non_english(apple_clean, 1)



1014  non-English rows removed.
6181  rows remaining.




Now, we will remove rows for all apps that aren't free.

In [96]:
apple_clean = remove_non_free(apple_clean, 1, 4)



2961  non-free rows removed.
3220  rows remaining.




Now, we will use the same functions to update the Android dataset:

In [97]:
android_clean = remove_duplicates(android_list[1:], 0, 3)

android_clean = remove_non_english(android_clean, 0)

android_clean = remove_non_free(android_clean, 0, 7)



1181  duplicate rows removed.
9660  rows remaining.




45  non-English rows removed.
9615  rows remaining.




750  non-free rows removed.
8865  rows remaining.




This is the frequency table for prime_genre in the Apple dataset:

In [98]:
display_table(apple_clean, 11)

Games : 1872
Entertainment : 254
Photo & Video : 160
Education : 118
Social Networking : 106
Shopping : 84
Utilities : 81
Sports : 69
Music : 66
Health & Fitness : 65
Productivity : 56
Lifestyle : 51
News : 43
Travel : 40
Finance : 36
Weather : 28
Food & Drink : 26
Reference : 18
Business : 17
Book : 14
Navigation : 6
Medical : 6
Catalogs : 4


This is the frequency table for Genres in the Android dataset:

In [99]:
display_table(android_clean, 9)

Tools : 749
Entertainment : 538
Education : 474
Business : 407
Productivity : 345
Lifestyle : 345
Finance : 328
Medical : 313
Sports : 307
Personalization : 294
Communication : 287
Action : 275
Health & Fitness : 273
Photography : 261
News & Magazines : 248
Social : 236
Travel & Local : 206
Shopping : 199
Books & Reference : 190
Simulation : 181
Dating : 165
Arcade : 164
Video Players & Editors : 157
Casual : 156
Maps & Navigation : 124
Food & Drink : 110
Puzzle : 100
Racing : 88
Role Playing : 83
Libraries & Demo : 83
Auto & Vehicles : 82
Strategy : 81
House & Home : 73
Weather : 71
Events : 63
Adventure : 60
Comics : 54
Beauty : 53
Art & Design : 53
Parenting : 44
Card : 40
Casino : 38
Trivia : 37
Educational;Education : 35
Board : 34
Educational : 33
Education;Education : 30
Word : 23
Casual;Pretend Play : 21
Music : 18
Racing;Action & Adventure : 15
Puzzle;Brain Games : 15
Entertainment;Music & Video : 15
Casual;Brain Games : 12
Casual;Action & Adventure : 12
Arcade;Action & Advent

This is the frequency table for Category in the Android dataset:

In [100]:
display_table(android_clean, 1)

FAMILY : 1676
GAME : 862
TOOLS : 750
BUSINESS : 407
LIFESTYLE : 346
PRODUCTIVITY : 345
FINANCE : 328
MEDICAL : 313
SPORTS : 301
PERSONALIZATION : 294
COMMUNICATION : 287
HEALTH_AND_FITNESS : 273
PHOTOGRAPHY : 261
NEWS_AND_MAGAZINES : 248
SOCIAL : 236
TRAVEL_AND_LOCAL : 207
SHOPPING : 199
BOOKS_AND_REFERENCE : 190
DATING : 165
VIDEO_PLAYERS : 159
MAPS_AND_NAVIGATION : 124
FOOD_AND_DRINK : 110
EDUCATION : 103
ENTERTAINMENT : 85
LIBRARIES_AND_DEMO : 83
AUTO_AND_VEHICLES : 82
HOUSE_AND_HOME : 73
WEATHER : 71
EVENTS : 63
PARENTING : 58
ART_AND_DESIGN : 57
COMICS : 55
BEAUTY : 53
1.9 : 1
